







































































































































# Long Call Option

### I. Strategy Overview

#### Payoff

![](long_call.jpg)

#### Features:
* Long Delta: option value increases as the underlying price goes up
* Short Theta: option value decreases as time decays
* Long volatility: option value increases as the implied volatility hikes

### II. Strategy Setup

In [ ]:
import sys  
sys.path.insert(0, 'C:\\Users\\chena\\Desktop\\Trading\\StrategicOptions\\')
import datetime
from Functions.mainprg import GetDays, GetPayOff, GetBackTesting, GetPriceHistory
import pandas as pd
import numpy as np
from datetime import timedelta

####  1. Input symbol of underlying

In [ ]:
Ticker = 'MSFT'
GetPriceHistory(Ticker).plot_price_history()

#### 2. Specify testing period and find out the expiration dates within

In [ ]:
Start_Date = datetime.datetime(2017, 1, 4)
End_Date = datetime.datetime(2017,5, 31)

expiry = GetDays(Ticker, Start_Date, End_Date, []).expiration_dates
expiry

#### 3. Choose option expiration and  enter dates

In [ ]:
entry_date = datetime.datetime(2018, 2, 5)
expiration = datetime.datetime(2018, 5, 18)

#### 4. Option Type & Strike Price Selection
* Change the strike price to see how it affects the strategy

In [ ]:
option_type = ['P', 'C']
action = ['S','S']
strike_price = [85, 85]  # Deep in the money
# strike_price = 55  # At the money
# strike_price = 65  # Far out of the money

### III. Visualization
#### 1. Generate options basic data
<font color='red'>POP calc and breakeven needs more thinking</font>

In [ ]:
payoff = GetPayOff(Ticker, strike_price, entry_date, expiration, option_type, action)
payoff.plot_payoff()


#### 3. Actual Price and Volatility Moves

In [ ]:
exit_date = datetime.datetime(2018, 5, 17)
bt1 = GetBackTesting(Ticker, strike_price, expiration, entry_date, exit_date, option_type, action)

#### 4. Returns

In [ ]:
bt1.plot_price_history()

In [ ]:
bt1.plot_option_pnl()

#### 5. Trade Journaling

In [ ]:
bt1.strategy_summary

### IV. Recurring Strategy Setup
Get in when:
Simplified
* 1st trading day of every month
* Buy next month's expiration

Realistic:
* IV < 0
* Expiration date < 60 & Expiration date > 60
* VIX < 20

In [ ]:
trading_days = GetDays(Ticker, Start_Date, End_Date, []).trading_days
trading_days['Year-Month'] = pd.to_datetime(trading_days['Date']).dt.to_period('M') + 1  # +1 to find the expiration date in the next month

dte_seeker = pd.merge(trading_days, expiry, on='Year-Month', how='left')
dte_seeker['DTE'] = (pd.to_datetime(dte_seeker['Expiration Date']) - pd.to_datetime(dte_seeker['Date'])).dt.days
dte_seeker.dropna(inplace=True)

#### 43 DTE example

In [ ]:
entry_dates = pd.to_datetime(dte_seeker[dte_seeker['DTE']==43]['Date'].reset_index(drop=True))
expiration_dates = pd.to_datetime(dte_seeker[dte_seeker['DTE']==43]['Expiration Date'].reset_index(drop=True))
exit_dates = entry_dates + timedelta(days=25)
expiration_dates = pd.to_datetime(expiration_dates)
# exit_dates = dte_seeker[(dte_seeker['DTE']==21)]['Date']
# exit_dates[exit_dates>entry_dates[0]].reset_index(drop=True)

In [ ]:
strategy_performance = pd.DataFrame([])
strat_sum_for_stats = []
strike_price = np.array(strike_price)
i = 0
for _ in entry_dates:
    bt = GetBackTesting(Ticker, strike_price, expiration_dates[i], entry_dates[i], exit_dates[i], option_type, action)
    strategy_performance = strategy_performance.append(bt.strategy_summary)
    strat_sum_for_stats.append(bt.strat_sum_for_stats)
    i += 1
    end_of_period_spot = bt.underlying_price_truncated_['close'][-1]
    strike_price = [end_of_period_spot-10, end_of_period_spot-5]
    
win_rate = "{:.2%}".format(strategy_performance['Win'].sum()/len(strategy_performance['Win']))
# Total_Cost = "${:,.2f}".format(strategy_performance['Cost Basis'].sum())

In [ ]:
win_rate

In [ ]:
strategy_performance

In [ ]:
strategy_performance['P&L']

In [ ]:
pnl_sum =strategy_performance['P&L'].str.strip('$').astype(float).sum()
print('Total Profit: ', pnl_sum)